In [90]:
import MetaTrader5 as mt5
import pandas as pd 
import numpy as np 
from scipy.stats import norm 
mt5.initialize()

True

In [91]:
def BlackScholes(r,S,K,T,sigma, tipoOpcao):
    # calcular o preco teórico pelo BS
    d1 = (np.log(S/K) + (r + sigma**2/2)*T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    # vou calcular apenas para o P (Put)
    tipoOpcao = 'P'
    if tipoOpcao.upper() == "C":
        precoTeoricoOpcao = S * norm.cdf(d1,0,1) - K * np.exp(-r*T) * norm.cdf(d2,0,1)
    else:
        precoTeoricoOpcao = K * np.exp(-r*T) * norm.cdf(-d2,0,1) - S * norm.cdf(-d1,0,1) 
    precoTeoricoOpcao = round(precoTeoricoOpcao,2)
    return precoTeoricoOpcao


In [117]:
def RolarTravaAltaPut(ativosLista, comprei, vendi, diasUteisParaVencer ):
    # [travaFeitaC (comprada) , travaFeitaV (vendida), travaNovaC, travaNovaV]
    # comprei: valor que comprei para montar TA    vendi: valor que vendi
    # vendi - comprei = spread
    travaFeitaC = mt5.symbol_info(ativosLista[0])
    travaFeitaV = mt5.symbol_info(ativosLista[1])
    travaNovaC  = mt5.symbol_info(ativosLista[2])
    travaNovaV  = mt5.symbol_info(ativosLista[3])
    # ativoBase = travaFeitaC.basis
    ativoBaseInfo = mt5.symbol_info(travaFeitaC.basis)

    # gerar matriz
    lin,col = (5,11)
    matriz = [[0 for i in range(col)] for j in range(lin)]
    ###########################################
    # coluna 0 - ativos
    cc = 0 
    matriz[0][cc] = ativoBaseInfo.name
    matriz[1][cc] = travaFeitaC.name 
    matriz[2][cc] = travaFeitaV.name
    matriz[3][cc] = 'dif'
    matriz[4][cc] = '%'

    # coluna 1 - strike feitos
    cc +=1 
    matriz[0][cc] = ativoBaseInfo.last
    matriz[1][cc] = travaFeitaC.option_strike
    matriz[2][cc] = travaFeitaV.option_strike
    matriz[3][cc] = travaFeitaV.option_strike - travaFeitaC.option_strike
    matriz[4][cc] = round((travaFeitaV.option_strike / ativoBaseInfo.last -1)*100, 2) 

    # coluna 2 - valores da trava feita
    cc +=1 
    matriz[0][cc] = '*'
    matriz[1][cc] = round(comprei,2)
    matriz[2][cc] = round(vendi,2)
    matriz[3][cc] = round(matriz[1][2] + matriz[2][2], 2)   # spread da TA

    # coluna - valores para desmontar a trava feita
    cc +=1 
    matriz[0][cc] = '*'
    matriz[1][cc] = round(travaFeitaC.bid,2)
    matriz[2][cc] = round(-travaFeitaV.ask,2)
    matriz[3][cc] = round(matriz[1][3] + matriz[2][3], 2)

   # coluna - B&S para desmontar
    cc +=1 
        
    r = 0.02                        # r = taxa de juros    
    S = ativoBaseInfo.last          # S = preço da ação 
    T = diasUteisParaVencer[0]/365               # T = tempo para vencer a opção
    sigma = 0.30                    # sigama = volatilidade
    K =  travaFeitaC.option_strike  # K = strike
    matriz[0][cc] = '*'
    matriz[1][cc] = BlackScholes(r,S,K,T,sigma, 'P')
    K =  travaFeitaV.option_strike  # K = strike
    matriz[2][cc] = -BlackScholes(r,S,K,T,sigma, 'P')
    matriz[3][cc] = round(matriz[1][cc] + matriz[2][cc], 2)

    # COLUNA - LAST - ultimos preços negociados nas opções da trava feita
    cc +=1 
    matriz[0][cc] ='*'
    matriz[1][cc] = travaFeitaC.last
    matriz[2][cc] = -travaFeitaV.last
    matriz[3][cc] = matriz[1][cc] + matriz[2][cc]
    matriz[4][cc] = '' 


    # coluna 4 - ativos novos
    cc +=1 
    matriz[0][cc] = '*'
    matriz[1][cc] = travaNovaC.name
    matriz[2][cc] = travaNovaV.name
    matriz[3][cc] = 'dif'
    matriz[4][cc] = '%'

    
    # coluna 5 - strikes novos
    cc +=1 
    matriz[0][cc] = '*'
    matriz[1][cc] = round(travaNovaC.option_strike,2)
    matriz[2][cc] = round(travaNovaV.option_strike,2)
    matriz[3][cc] = matriz[2][cc] - matriz[1][cc]
    matriz[4][cc] = round((travaFeitaV.option_strike / ativoBaseInfo.last -1)*100, 2) 

    # coluna 6 - valores para montar nova trava
    cc +=1 
    matriz[0][cc] = '*'
    matriz[1][cc] = -travaNovaC.ask
    matriz[2][cc] = travaNovaV.bid
    if matriz[2][cc] == 0:
        fff = input('Entre o valor do Bid ' + travaNovaV.name + ': ')
        matriz[2][cc] = float(fff)
    matriz[3][cc] = matriz[1][cc] + matriz[2][cc]

    # COLUNA - Black & Scholes
    cc +=1 

    r = 0.02                        # r = taxa de juros    
    S = ativoBaseInfo.last          # S = preço da ação 
    T = diasUteisParaVencer[1]/365               # T = tempo para vencer a opção
    sigma = 0.30                    # sigama = volatilidade
    K = travaNovaC.option_strike  # K = strike
    matriz[0][cc] ='.'
    matriz[1][cc] = -BlackScholes(r,S,K,T,sigma, 'P')
    K = travaFeitaV.option_strike  # K = strike
    matriz[2][cc] = BlackScholes(r,S,K,T,sigma, 'P')
    matriz[3][cc] = matriz[1][cc] + matriz[2][cc]
    matriz[4][cc] = '' 

    # COLUNA - LAST - ultimos preços negociados nas opções da trava a fazer
    cc +=1 
    matriz[0][cc] ='*'
    matriz[1][cc] = -travaNovaC.last
    matriz[2][cc] = travaNovaV.last
    matriz[3][cc] = matriz[1][cc] + matriz[2][cc]
    matriz[4][cc] = '' 


    columns = ['Ativo', '$$$', '$ Montei', '$ Desmontar' , 'B&S' , '$ last TA feita', 'ativo', 'Strike Novo', 'Montar Novo'  , 'B&S', '$ last TA Nova']
    df = pd.DataFrame(matriz, columns=columns)
    print(df)

    nome_arq = 'RolarTravaAltaPut_' + ativoBaseInfo.name  + '.xlsx'
    df.to_excel(nome_arq)


In [116]:
diasUteisParaVencer = [4,29]   # quantos dias faltam para vencer as opções montadas e a montar, respectivamente
RolarTravaAltaPut(['BBSES220', 'BBSES235', 'BBSET220', 'BBSET235'], -0.15, 0.4, diasUteisParaVencer)

      Ativo    $$$ $ Montei $ Desmontar   B&S $ last TA feita     ativo  \
0     BBSE3  22.60        *          **    bs               .       ***   
1  BBSES220  22.00    -0.15        0.06  0.08            0.11  BBSET220   
2  BBSES235  23.50      0.4        -0.0 -0.93           -0.91  BBSET235   
3       dif   1.50     0.25        0.06 -0.85            -0.8       dif   
4         %   3.98        0           0     0                         %   

  Strike Novo Montar Novo   B&S $ last TA Nova  
0        ****        ****     .              .  
1        22.0        -0.5 -0.48          -0.39  
2        23.5        0.95  1.28           1.16  
3         1.5        0.45   0.8           0.77  
4        3.98           0                       
